In [2]:
import os
import json
import pdfplumber
import spacy
import re
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# Cargar modelos
nlp = spacy.load("es_core_news_sm")
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
ner_pipeline = pipeline("ner", model="PlanTL-GOB-ES/roberta-base-bne", aggregation_strategy="simple")

# Directorios de almacenamiento
DATA_DIR = "data"
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

# 🔹 1. Extraer texto de PDF
def extraer_texto_pdf(ruta_pdf):
    texto_completo = ""
    with pdfplumber.open(ruta_pdf) as pdf:
        for pagina in pdf.pages:
            texto_pagina = pagina.extract_text(x_tolerance=1, y_tolerance=3)
            if texto_pagina:
                texto_completo += texto_pagina.strip() + "\n\n"  # Agregar doble salto de línea para separar páginas correctamente
    return texto_completo.strip()

# 🔹 2. Segmentar texto en párrafos
def segmentar_en_parrafos(texto):
 #   parrafos = re.split(r"\n\s*\n", texto.strip())  # Separar por dobles saltos de línea
 #   return [p.strip() for p in parrafos if len(p.strip()) > 30]  # Evitar fragmentos cortos
 #   Versión 1 -> codigo anterior -> Separa por páginas -> cambiamos a nueva versión
    """
    Separa el texto en párrafos usando:
    - Dobles saltos de línea
    - Punto seguido de salto de línea y una mayúscula (inicio de un nuevo párrafo)
    """
    # Normalizar saltos de línea
    texto = re.sub(r'\n+', '\n', texto)  # Reemplazar múltiples saltos por uno solo

    # Separar por dobles saltos de línea o punto seguido de un salto de línea y mayúscula
    parrafos = re.split(r"\n\s*\n|(?<=\.)\n(?=[A-Z])", texto)

    # Filtrar fragmentos muy cortos
    parrafos = [p.strip() for p in parrafos if len(p.strip()) > 30]

    return parrafos

# 🔹 3. Extraer entidades clave (NER)
import spacy

# Cargar modelo NER en español más robusto
# nlp = spacy.load("es_core_news_sm") -> con este modelo los resultados son muy malos

# El archivo generado con las entidades clave (NER) contiene resultados erróneos. 📉

# 📌 Problemas observados:
# ✅ Las entidades detectadas están mal segmentadas.
# ✅ Muchas entidades parecen fragmentos de palabras en lugar de términos completos.
# ✅ Los tipos de entidades (LABEL_0, LABEL_1) no tienen significado útil.
# ✅ Errores en la tokenización, con cortes extraños en palabras.
# Análisis de las Causas del Problema
# 1️⃣ El modelo NER usado (PlanTL-GOB-ES/roberta-base-bne) no está funcionando bien con el texto extraído.
# 2️⃣ El texto del documento podría estar mal segmentado, afectando la detección de entidades.
# 3️⃣ El modelo NER usado requiere una preprocesamiento diferente (quizás eliminar saltos de línea, caracteres especiales, etc.).
# 4️⃣ Las entidades detectadas no están bien clasificadas en PERSON, ORG, LOC, etc., lo que indica un problema con el pipeline("ner").
# Realizados los cambios
# Cargar modelo NER en español más robusto
# limpiar_texto + nlp = spacy.load("es_core_news_sm")
# Problemas Detectados
# ✅ Clasificación incorrecta de entidades → PER (Persona) asignado a términos como "Model Report" o "Make".
# ✅ Demasiadas entidades MISC sin valor informativo.
# ✅ Entidades mal segmentadas y con palabras fragmentadas.
# ✅ Poca detección de entidades de tipo ORG (Organización) o LOC (Ubicación).

# 2️⃣ Si los resultados aún tienen problemas, considerar otro modelo (es_core_news_lg).

# nlp = spacy.load("es_core_news_lg") -> no funciona bien es un modelo para texto en español

nlp = spacy.load("en_core_web_lg")


def limpiar_texto(texto):
    """
    Normaliza el texto eliminando saltos de línea innecesarios y caracteres extraños.
    """
    texto = texto.replace("\n", " ")  # Reemplazar saltos de línea con espacios
    texto = re.sub(r'\s+', ' ', texto).strip()  # Eliminar espacios dobles
    return texto


def extraer_entidades(parrafo):
 #   entidades = ner_pipeline(parrafo)
 #   return [{"texto": ent["word"], "tipo": ent["entity_group"]} for ent in entidades]
 
 #  Extrae entidades clave (NER) usando spaCy.
 
    parrafo = limpiar_texto(parrafo)  # Limpiar el texto antes de pasarlo al NER
    doc = nlp(parrafo)
    entidades = [
        {"texto": ent.text, "tipo": ent.label_}
        for ent in doc.ents
        if ent.label_ in ["PER", "ORG", "LOC", "MISC", "DATE"]
    ]  # Filtrar solo entidades útiles

    return entidades

# 🔹 4. Calcular embeddings
def calcular_embeddings(secciones):
    return embedding_model.encode(secciones)

# 🔹 5. Procesar un documento
def procesar_documento(ruta_pdf):
    # Obtener nombre del archivo sin extensión
    nombre_archivo = os.path.splitext(os.path.basename(ruta_pdf))[0]

    print(f"📂 Procesando: {nombre_archivo}")

    # Extraer texto
    texto = extraer_texto_pdf(ruta_pdf)
    if not texto:
        print(f"⚠️ No se pudo extraer texto de {nombre_archivo}")
        return

    # Guardar texto extraído en un archivo .txt
    ruta_txt = os.path.join(DATA_DIR, f"{nombre_archivo}.txt")
    with open(ruta_txt, "w", encoding="utf-8") as f:
        f.write(texto)
    print(f"📝 Texto extraído guardado en: {ruta_txt}")


    # Segmentar en secciones
    secciones = segmentar_en_parrafos(texto)
    print(f"🔹 Se detectaron {len(secciones)} secciones en {nombre_archivo}")

    # Calcular embeddings de cada sección
    embeddings = calcular_embeddings(secciones)

    # Extraer entidades clave de cada sección
    entidades_totales = {}
    for idx, seccion in enumerate(secciones):
        entidades = extraer_entidades(seccion)
        entidades_totales[f"seccion_{idx+1}"] = entidades

    # Guardar embeddings en archivo JSON
    embeddings_json = {
        "archivo": nombre_archivo,
        "secciones": [{"id": f"seccion_{i+1}", "texto": sec, "embedding": emb.tolist()} for i, (sec, emb) in enumerate(zip(secciones, embeddings))]
    }
    
    with open(os.path.join(DATA_DIR, f"{nombre_archivo}_embeddings.json"), "w", encoding="utf-8") as f:
        json.dump(embeddings_json, f, ensure_ascii=False, indent=4)

    # Guardar entidades en archivo JSON
    entidades_json = {
        "archivo": nombre_archivo,
        "entidades": entidades_totales
    }

    with open(os.path.join(DATA_DIR, f"{nombre_archivo}_entidades.json"), "w", encoding="utf-8") as f:
        json.dump(entidades_json, f, ensure_ascii=False, indent=4)

    print(f"✅ Procesamiento de {nombre_archivo} completado. Datos almacenados en {DATA_DIR}/")

# 🔹 6. Procesar documentos de una carpeta
if __name__ == "__main__":
    directorio_documentos = "./documentos"
    
    for archivo in os.listdir(directorio_documentos):
        if archivo.endswith(".pdf"):
            procesar_documento(os.path.join(directorio_documentos, archivo))


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


📂 Procesando: 0e21835a42a6df2405496f62647058ff855743c1
📝 Texto extraído guardado en: data\0e21835a42a6df2405496f62647058ff855743c1.txt
🔹 Se detectaron 21 secciones en 0e21835a42a6df2405496f62647058ff855743c1
✅ Procesamiento de 0e21835a42a6df2405496f62647058ff855743c1 completado. Datos almacenados en data/
